<a href="https://colab.research.google.com/github/SeHwanJoo/2021_cau_oss_hackathon/blob/main/hackathon_team11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [ ]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

170508288/170498071 [==============================] - 3s 0us/step


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10_1/v6/1.1.0.incompleteX5CN0M/cifar10_1-test.tfrecord


  0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset cifar10_1 downloaded and prepared to /root/tensorflow_datasets/cifar10_1/v6/1.1.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


# **2. 데이터 전처리**



In [ ]:
import os, sys 
from google.colab import drive

drive.mount('/content/mnt', force_remount=True)

Mounted at /content/mnt


In [ ]:
import numpy as np
# 데이터 normalization

# x_train - (50000, 32, 32, 3)
# y_train - (50000,10)

def normalization(train_images, test_images1, test_images2):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images1 = (test_images1 - mean) / (std + 1e-7)
    test_images2 = (test_images2 - mean) / (std + 1e-7)
    return train_images, test_images1, test_images2

In [ ]:
# normalization 적용

x_train_after, x_test1_after, x_test2_after = normalization(x_train, x_test1, x_test2)

# **3. 모델 생성**

In [ ]:
# 패키지 install

!pip install -U git+https://github.com/GdoongMathew/EfficientNetV2 --no-deps

  Cloning https://github.com/GdoongMathew/EfficientNetV2 to /tmp/pip-req-build-uq7yiypw
  Running command git clone -q https://github.com/GdoongMathew/EfficientNetV2 /tmp/pip-req-build-uq7yiypw
  Created wheel for efficientnetv2: filename=efficientnetv2-0.0.1-py3-none-any.whl size=12269 sha256=4dcc55cd93046224db6284277143d818681e40443719b91c753934e8008341b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-skpi8o6f/wheels/8d/84/21/b4cb691850da4bcb1d607b46a9ef56ce58e04d74c447948c2a
Successfully built efficientnetv2


In [ ]:
# CONFIG 설정

CONFIG = {
    'name': 'new_setting',
    'epoch': 100,
    'batch_size' : 128,
    'activation': 'sigmoid', # sigmoid, softmax 
    'lr':0.1, 
}

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

# Augmentation layer 

data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

In [ ]:
pj_name = CONFIG['name']
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'/content/drive/MyDrive/body_tumor/hyelin/ckpt/{pj_name}_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

In [ ]:
def build_optimizer(learning_rate=0.1, momentum=0.9):

    # reducelronplateau / cosineannealing / step / ...
    learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
        [500, 16000, 24000],
        [learning_rate / 10., learning_rate, learning_rate / 10., learning_rate / 100.])


    # SGD / Adam / Adamw
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

    return optimizer

In [ ]:
import efficientnetv2
from efficientnetv2.utils import DENSE_KERNEL_INITIALIZER, CONV_KERNEL_INITIALIZER


input_x = tf.keras.layers.Input(shape=(None, None, 3)) # input shape 정의

x = data_augmentation(input_x) # augmentation layer 추가

effnet = efficientnetv2.EfficientNetV2_S(weights='imagenet',
                                         include_top = False,
                                         input_tensor = input_x
                                         ) # efficientnetv2 모델 정의
x = effnet(x, training=True)

output_x = tf.keras.layers.Dense(10,
                                activation = CONFIG['activation'],
                                kernel_initializer = DENSE_KERNEL_INITIALIZER,
                                dtype = tf.float32, name = 'final_dense')(x) #classification head 추가

model = tf.keras.Model(inputs=input_x, outputs=output_x)

model.compile(
    optimizer = build_optimizer(),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
    #loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']) # 모델 complie

model.summary()

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, None, None, 3)]   0         
_________________________________________________________________
sequential_1 (Sequential)    (None, None, None, 3)     0         
_________________________________________________________________
efficientnetv2_s (Functional (None, 1280)              20331360  
_________________________________________________________________
final_dense (Dense)          (None, 10)                12810     
Total params: 20,344,170
Trainable params: 20,190,298
Non-trainable params: 153,872
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, batch_size = CONFIG['batch_size'], epochs = CONFIG['epoch'], shuffle=True, callbacks=[cp_callback], validation_data=(x_test2, y_test2))

# **4. 모델 저장**

In [ ]:
save_path = '/content/'
team_name = 'team11'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [ ]:
save_path = '/content/'
team_name = 'team11'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test1, y_test1)
model.evaluate(x_test2, y_test2)